In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Flatten, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, ReLU, Dense, Input, Dropout
from tensorflow.keras.models import Model
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.utils import get_custom_objects
import cvzone

In [ ]:
cvzone.putTextRect

In [2]:
def train_dg(path):

    train = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip=True,
    rescale=1./255,
    )

    train = train.flow_from_directory(
    directory=path,
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
    )
    return train

In [3]:
def val_dg(path):

    test = ImageDataGenerator(
        rescale=1./255
    )

    test = test.flow_from_directory(
    directory=path,
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
    )
    return test

In [4]:
train_datagen = train_dg(r"E:\archive\data\train")

Found 2942 images belonging to 3 classes.


In [5]:
test_datagen = val_dg(r"E:\archive\data\val")

Found 737 images belonging to 3 classes.


In [6]:
print(train_datagen.class_indices)

{'Brown_rust': 0, 'Healthy': 1, 'Yellow_rust': 2}


In [7]:
def conv(x, filters, kernel_size, strides):
    x = Conv2D( filters = filters, kernel_size = kernel_size, strides= strides, padding='same', activation="relu", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def depv(x, filters, kernel_size, strides):
    x = DepthwiseConv2D(kernel_size=kernel_size, strides=strides,  padding='same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters = filters, kernel_size = (1, 1), strides= 1, padding='same', activation="relu", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

In [8]:
def model(x):
    x = conv(x, 32, 3, 2)
    x = depv(x, 64, 3, 1)
    x = depv(x, 128, 3, 2)
    x = depv(x, 128, 3, 1)
    x = depv(x, 256, 3, 2)
    x = depv(x, 256, 3, 1)
    x = depv(x, 512, 3, 2)
    for i in range(5):
        x = depv(x, 512, 3, 1)
    x = depv(x, 1024, 3, 2)
    x = depv(x, 1024, 3, 2)
    # x = Flatten()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(units=3, activation="softmax", use_bias=False)(x)
    return x

In [9]:
input = Input(shape=(224, 224, 3))
output = model(input)
cnn = Model(inputs=input, outputs=output)

In [10]:
cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      864       
                                                                 
 batch_normalization (Batch  (None, 112, 112, 32)      128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 112, 112, 32)      288       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 32)      128   

In [11]:
cnn.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

In [12]:
callback=[
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.2, verbose=1)
]

In [ ]:
cnn.fit(x=train_datagen, validation_data=test_datagen, epochs=30, batch_size=32, callbacks=[callback])

Epoch 1/30


92/92 [==============================] - 258s 3s/step - loss: 0.8070 - accuracy: 0.6322 - val_loss: 1.0995 - val_accuracy: 0.3148 - lr: 0.0010
Epoch 2/30
92/92 [==============================] - 182s 2s/step - loss: 0.4247 - accuracy: 0.8389 - val_loss: 1.0996 - val_accuracy: 0.3148 - lr: 0.0010
Epoch 3/30
92/92 [==============================] - 167s 2s/step - loss: 0.3591 - accuracy: 0.8725 - val_loss: 1.0996 - val_accuracy: 0.3148 - lr: 0.0010
Epoch 4/30
92/92 [==============================] - ETA: 0s - loss: 0.3075 - accuracy: 0.8919
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
92/92 [==============================] - 189s 2s/step - loss: 0.3075 - accuracy: 0.8919 - val_loss: 1.2471 - val_accuracy: 0.3066 - lr: 0.0010
Epoch 5/30
84/92 [==========================>...] - ETA: 13s - loss: 0.2142 - accuracy: 0.9226

In [13]:
cnn.save("mobilenet.h5")

C:\Users\tharu\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
test_image =image.load_img(r"E:\wheat disease\data\val\Healthy\Healthy057.jpg", target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
print(test_image.shape)
result = model.predict(test_image)
print(result)

(1, 224, 224, 3)
1/1 [==============================] - 1s 528ms/step
[[0. 1. 0.]]


In [14]:
print(test_datagen.class_indices)

{'Brown_rust': 0, 'Healthy': 1, 'Yellow_rust': 2}
